Test residuals in the data with additional masking of DEPTH

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import healpy as hp
import fitsio as ft
import pandas as pd
from utils import hpix2radec
from nnbar import NNBAR

In [ ]:
df = pd.read_hdf('/Volumes/TimeMachine/data/DR7/DR7meta.h5')
depth_g = df['depth_g_total'].values

galmap = hp.read_map('/Volumes/TimeMachine/data/DR7/eBOSS.ELG.NGC.DR7.cut.hp256.fits', verbose=False)
fracmap = hp.read_map('/Volumes/TimeMachine/data/DR7/frac.hp.256.fits', verbose=False)
mask = hp.read_map('/Volumes/TimeMachine/data/DR7/mask_data_mock.cut.hp.256.fits', verbose=False) > 0
wnn = hp.read_map('/Volumes/TimeMachine/data/DR7/results/regression/nn_p/nn-weights.hp256.fits', verbose=False)
wnn2 = hp.read_map('/Volumes/TimeMachine/data/DR7/results_referee/nn_p/nn-weights.hp256.fits', verbose=False)

In [ ]:
chi2 = lambda y, ye:(((y-1)/ye)**2).sum()

def run(maski, wnni=None):    
    
    # get mean density
    ndepth = NNBAR(galmap, fracmap, maski, depth_g, selection=wnni)
    ndepth.run()

    
    y = ndepth.output['nnbar']
    x = (ndepth.output['bin_edges'][:-1])#+ndepth.output['bin_edges'][1:])
    ye = ndepth.output['nnbar_err']

    nan = np.isnan(y) | np.isnan(ye)
    if nan.sum()!=0:
        print(y, nan.sum())
        y = y[~nan]
        x = x[~nan]
        ye = ye[~nan]


    fig, ax1 = plt.subplots()

    ax2 = ax1.twinx()
    ax1.errorbar(x, y, yerr=ye, marker='o', color='b', ls='none')
    for xmin in x[:-3]:
        chi2v = chi2(y[x>xmin], ye[x>xmin])
        print(xmin, chi2v)
        ax2.scatter(xmin, chi2v, 
                    color='darkorange', marker='s', facecolors='none')

    ax1.set_ylim(0.8, 1.2)
    ax1.axhline(1, ls=':', color='blue')
    #ax2.axhline(50, ls='--', color='orange')
    #ax2.text(24.6, 52, '95% Null mocks', color='orange')

    ax1.set_xlabel('Depth_g')
    ax1.set_ylabel('N/Nbar', color='b')
    ax2.set_ylim(-5, 85)
    ax2.set_ylabel('Chi2(>depth_g)', color='darkorange', fontweight='bold')

In [ ]:
run(mask, None)

In [ ]:
run(mask, wnn)

In [ ]:
run(mask, wnn2)

In [ ]:
run(mask & (depth_g >= 23.95), wnn)

In [ ]:
1- (depth_g[mask] >= 23.95).mean()

In [ ]:
(depth_g[mask] < 23.9).mean()

In [ ]:
ra, dec = hpix2radec(256, np.argwhere(mask).flatten())

In [ ]:
depth_g[mask].max()

In [ ]:
plt.figure(figsize=(12, 4))
plt.hist(depth_g[mask], 
         bins=65, range=(22.7, 44.6),
         histtype='step')
plt.yscale('log')
plt.xticks(np.linspace(22, 45, 16))
plt.xlabel('depth g')

In [ ]:
cmap = plt.cm.get_cmap('jet_r', 10)
plt.scatter(ra, dec, 1, c=depth_g[mask], marker='H',  
            cmap=cmap, vmin=24, vmax=26)
plt.colorbar()